In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/df-aneka-industri-2021-multinomial/dataset financial data ratio aneka industri 2021 (1).csv


***
# Multinomial Logistic Regression NPL Score -  Emiten Aneka Industri sub sektor Mesin dan Komponen Industri  Q4 2021 -  HAJAR HANIFAH 
***

Tujuan penelitian ini adalah melakukan Multinomial Logistic Regression, mencari NPL Score terhadap data seluruh emiten saham di sektor aneka industri untuk sub sektor Mesin dan Komponen Industri Q4 2021. 
Adapun beberapa step yang akan dilakukan adalah 
1. Import Libraries
2. Exploratory Data Analysis
    - Data Understanding
    - Understanding the Table
    - Load Data
    - Modeling
3. Multinomial logistic regression with scikit-learn
    - Data Preparation
    - Fit the Model
    - Standardize the Data
    - Model Validation
    - Probabilitas
 4. Conclution

menggunakan rasio :

- X1. DER = Total Debt / Total Equity
- X2. Return on Asset = Net Income / total Asset
- X3. Sales Growth = (Sales 2021 - sales 2020)/ sales 2020
- X4. Others
- X5 : others
- Y. Performance / Category = Using Clustering result yang hasilnya didapatkan dari [Clustering Sektor Industri - Aneka Industri 2021](https://www.kaggle.com/code/hajarhanifah/clustering-sektor-industri-hajar-hanifah/edit/run/93274745)

***
# 1. Import Libraries
***

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import matplotlib.pyplot as plt



import statsmodels.api as sm

# readable figures
pd.set_option('float_format', '{:f}'.format)

***
# 2. Exploratory Data Analysis
***


# 2.1 Data Understanding

Data yang diambil merupakan data statictical financial ratio Q4 2021 dan 2020 untuk seluruh emiten di sektor Industri.

Data di dapatkan dari [IDX - Laporan Statistic Financial Data Ratio](https://www.idx.co.id/data-pasar/laporan-statistik/digital-statistic-beta/financial-data-ratio?q=eyJ5ZWFyIjoiMjAyMSIsIm1vbnRoIjpudWxsLCJxdWFydGVyIjo0LCJ0eXBlIjoieWVhcmx5In0=)


# 2.2 Understanding The Table

* **Emiten :** Pihak yang melakukan penawaran umum, yaitu penawaran efek yang dilakukan oleh emiten untuk menjual efek kepada masyarakat berdasarkan tata cara yang diatur dalam peraturan undang-undang yang berlaku. 
* **Sales Growth :** Kenaikan jumlah penjualan dari tahun ke tahun atau dari waktu ke waktu.
* **ROA (Return of Asset) :** Indikator untuk menunjukkan seberapa untuk sebuah perusahaan dibandingkan dengan total asetnya. 
* **DER (Debt to Equity Ratio) :** Rasio hutang terhadap ekuitas atau rasio keuangan yang membandingkan jumlah hutang dengan ekuitas

Keterangan rasio yang akan digunakan : 
- **DER =** Total Uang / Total Ekuitas
- **Return on Asset =** Laba setelah pajak / Total Asset 
- **Sales Growth =** (penjualan periode 2021 - penjualan periode 2020)/ penjualan periode 2020

# 2.3 Load Data

In [3]:
df = pd.read_csv('../input/df-aneka-industri-2021-multinomial/dataset financial data ratio aneka industri 2021 (1).csv')
df.head()

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.060000,0.330000,0.130000,0.120000,1
1,2,Industrial Machinery & Components,PT Ateliers Mecaniques D Indonesie Tbk.,AMIN,1.140000,-0.050000,0.410000,-0.120000,-0.130000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.050000,0.030000,0.070000,0.130000,1
3,4,Industrial Machinery & Components,PT Arkha Jayanti Persada Tbk.,ARKA,3.970000,-0.020000,0.220000,-0.080000,-0.160000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.210000,0.170000,0.320000,0.250000,4


In [4]:
df.describe()

,No,DER,ROA,Sales Growth,ROE,NPM,Category
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,15.000000,1.897586,0.025517,0.304483,0.021379,-0.053448,1.517241
std,8.514693,3.366741,0.103909,0.431456,0.241568,0.420300,1.121883
min,1.000000,-2.860000,-0.320000,-0.280000,-0.930000,-1.990000,0.000000
25%,8.000000,0.510000,-0.020000,0.030000,-0.080000,-0.050000,1.000000
50%,15.000000,0.880000,0.020000,0.220000,0.040000,0.040000,1.000000
75%,22.000000,1.910000,0.060000,0.410000,0.130000,0.130000,1.000000
max,29.000000,16.330000,0.310000,1.440000,0.520000,0.400000,4.000000


In [5]:
display(df)

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.060000,0.330000,0.130000,0.120000,1
1,2,Industrial Machinery & Components,PT Ateliers Mecaniques D Indonesie Tbk.,AMIN,1.140000,-0.050000,0.410000,-0.120000,-0.130000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.050000,0.030000,0.070000,0.130000,1
3,4,Industrial Machinery & Components,PT Arkha Jayanti Persada Tbk.,ARKA,3.970000,-0.020000,0.220000,-0.080000,-0.160000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.210000,0.170000,0.320000,0.250000,4
5,6,Industrial Machinery & Components,Cahayaputra Asa Keramik Tbk,CAKK,0.880000,0.030000,0.300000,0.060000,0.070000,1
6,7,Industrial Machinery & Components,Communication Cable Systems Indonesia Tbk,CCSI,0.310000,0.110000,0.790000,0.140000,0.170000,4
7,8,Industrial Machinery & Components,Citatah Tbk,CTTH,2.280000,-0.050000,-0.170000,-0.160000,-0.540000,1
8,9,Industrial Machinery & Components,Hexindo Adiperkasa Tbk,HEXA,1.380000,0.120000,0.680000,0.290000,0.210000,4
9,10,Industrial Machinery & Components,Sumi Indo Kabel Tbk,IKBI,0.670000,0.010000,0.670000,0.010000,0.010000,1


***
# 3. Multinomial logistic regression with scikit-learn
***

- assign predictors dan kriteria untuk setiap objek
- split x dan Y
- Y = categori
- buat variabel training dan testing

In [6]:
display(df)

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.060000,0.330000,0.130000,0.120000,1
1,2,Industrial Machinery & Components,PT Ateliers Mecaniques D Indonesie Tbk.,AMIN,1.140000,-0.050000,0.410000,-0.120000,-0.130000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.050000,0.030000,0.070000,0.130000,1
3,4,Industrial Machinery & Components,PT Arkha Jayanti Persada Tbk.,ARKA,3.970000,-0.020000,0.220000,-0.080000,-0.160000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.210000,0.170000,0.320000,0.250000,4
5,6,Industrial Machinery & Components,Cahayaputra Asa Keramik Tbk,CAKK,0.880000,0.030000,0.300000,0.060000,0.070000,1
6,7,Industrial Machinery & Components,Communication Cable Systems Indonesia Tbk,CCSI,0.310000,0.110000,0.790000,0.140000,0.170000,4
7,8,Industrial Machinery & Components,Citatah Tbk,CTTH,2.280000,-0.050000,-0.170000,-0.160000,-0.540000,1
8,9,Industrial Machinery & Components,Hexindo Adiperkasa Tbk,HEXA,1.380000,0.120000,0.680000,0.290000,0.210000,4
9,10,Industrial Machinery & Components,Sumi Indo Kabel Tbk,IKBI,0.670000,0.010000,0.670000,0.010000,0.010000,1


In [7]:
x = df.drop(['Category','No','Industri','Company','Emiten'], axis=1)
x.head()

,DER,ROA,Sales Growth,ROE,NPM
0,1.320000,0.060000,0.330000,0.130000,0.120000
1,1.140000,-0.050000,0.410000,-0.120000,-0.130000
2,0.490000,0.050000,0.030000,0.070000,0.130000
3,3.970000,-0.020000,0.220000,-0.080000,-0.160000
4,0.510000,0.210000,0.170000,0.320000,0.250000


In [8]:
y = df['Category']
y.head()

0    1
1    1
2    1
3    1
4    4
Name: Category, dtype: int64

# 3.2 Standardize the Data

In [9]:
from sklearn.preprocessing import StandardScaler 
from sklearn import preprocessing
x = preprocessing.StandardScaler().fit(x).transform(x)

In [10]:
display(df)

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.060000,0.330000,0.130000,0.120000,1
1,2,Industrial Machinery & Components,PT Ateliers Mecaniques D Indonesie Tbk.,AMIN,1.140000,-0.050000,0.410000,-0.120000,-0.130000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.050000,0.030000,0.070000,0.130000,1
3,4,Industrial Machinery & Components,PT Arkha Jayanti Persada Tbk.,ARKA,3.970000,-0.020000,0.220000,-0.080000,-0.160000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.210000,0.170000,0.320000,0.250000,4
5,6,Industrial Machinery & Components,Cahayaputra Asa Keramik Tbk,CAKK,0.880000,0.030000,0.300000,0.060000,0.070000,1
6,7,Industrial Machinery & Components,Communication Cable Systems Indonesia Tbk,CCSI,0.310000,0.110000,0.790000,0.140000,0.170000,4
7,8,Industrial Machinery & Components,Citatah Tbk,CTTH,2.280000,-0.050000,-0.170000,-0.160000,-0.540000,1
8,9,Industrial Machinery & Components,Hexindo Adiperkasa Tbk,HEXA,1.380000,0.120000,0.680000,0.290000,0.210000,4
9,10,Industrial Machinery & Components,Sumi Indo Kabel Tbk,IKBI,0.670000,0.010000,0.670000,0.010000,0.010000,1


In [11]:
trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.3)

Diketahui :
- training : 70%
- testing 30%

# 3.3 Fit The Model

In [12]:
log_reg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
log_reg.fit(trainX, trainY)
y_pred = log_reg.predict(testX)

# 3.4 Model Validation

**Print akurasi dan error rate**

In [13]:
print('Accuracy: {:.2f}'.format(accuracy_score(testY, y_pred)))
print('Error rate: {:.2f}'.format(1 - accuracy_score(testY, y_pred)))

Accuracy: 0.67
Error rate: 0.33


# 3.4 Probabilitas

In [14]:
probability = log_reg.predict_proba(testX)
probability

array([[8.09970098e-01, 1.69698874e-01, 2.03310276e-02],
       [9.94738866e-01, 2.40346435e-03, 2.85766916e-03],
       [9.27941739e-01, 1.60561120e-02, 5.60021489e-02],
       [9.47340896e-01, 5.88507409e-03, 4.67740300e-02],
       [9.99420213e-01, 5.53101897e-04, 2.66852514e-05],
       [9.12600169e-01, 1.12015995e-02, 7.61982315e-02],
       [9.99914270e-01, 8.17247043e-05, 4.00509322e-06],
       [9.36032599e-01, 2.15381728e-02, 4.24292283e-02],
       [9.59193722e-01, 1.51652919e-02, 2.56409861e-02]])

- Setiap kolom merepresentasikan class
- class dengan probabilitas paling tinggi = output dari predicted class


In [15]:
df1 = pd.DataFrame(log_reg.predict_proba(testX), columns=log_reg.classes_)
df1.head()

,1,2,4
0,0.809970,0.169699,0.020331
1,0.994739,0.002403,0.002858
2,0.927942,0.016056,0.056002
3,0.947341,0.005885,0.046774
4,0.999420,0.000553,0.000027


In [16]:
display(df1)

,1,2,4
0,0.809970,0.169699,0.020331
1,0.994739,0.002403,0.002858
2,0.927942,0.016056,0.056002
3,0.947341,0.005885,0.046774
4,0.999420,0.000553,0.000027
5,0.912600,0.011202,0.076198
6,0.999914,0.000082,0.000004
7,0.936033,0.021538,0.042429
8,0.959194,0.015165,0.025641


**Sum / jumlah probabilitas harus selalu 1**

In [17]:
df1['sum'] = df1.sum(axis=1)
df1.head()

,1,2,4,sum
0,0.809970,0.169699,0.020331,1.000000
1,0.994739,0.002403,0.002858,1.000000
2,0.927942,0.016056,0.056002,1.000000
3,0.947341,0.005885,0.046774,1.000000
4,0.999420,0.000553,0.000027,1.000000


**Lihat Predicted class**

In [18]:
df1['predicted_class'] = y_pred
df1.head()

,1,2,4,sum,predicted_class
0,0.809970,0.169699,0.020331,1.000000,1
1,0.994739,0.002403,0.002858,1.000000,1
2,0.927942,0.016056,0.056002,1.000000,1
3,0.947341,0.005885,0.046774,1.000000,1
4,0.999420,0.000553,0.000027,1.000000,1


**Actual Class**

In [19]:
df1['actual_class'] = testY.to_frame().reset_index().drop(columns='index')
df1.head()

,1,2,4,sum,predicted_class,actual_class
0,0.809970,0.169699,0.020331,1.000000,1,2
1,0.994739,0.002403,0.002858,1.000000,1,1
2,0.927942,0.016056,0.056002,1.000000,1,1
3,0.947341,0.005885,0.046774,1.000000,1,1
4,0.999420,0.000553,0.000027,1.000000,1,0


Dari data diatas, kita dapat melihat hasil dari prediksi setiap class. Untuk memudahkan dalam membaca prediksi, lakukan konversi kelas prediksi (prediction class) dan actual class menggunakan encoder label dari scikit-learn.

In [20]:
le = preprocessing.LabelEncoder()

df1['label_pred'] = le.fit_transform(df1['predicted_class'])
df1['label_actual'] = le.fit_transform(df1['actual_class'])
df1.head()

,1,2,4,sum,predicted_class,actual_class,label_pred,label_actual
0,0.809970,0.169699,0.020331,1.000000,1,2,0,2
1,0.994739,0.002403,0.002858,1.000000,1,1,0,1
2,0.927942,0.016056,0.056002,1.000000,1,1,0,1
3,0.947341,0.005885,0.046774,1.000000,1,1,0,1
4,0.999420,0.000553,0.000027,1.000000,1,0,0,0


In [21]:
targets = df1['predicted_class']   
integerEncoded = le.fit_transform(targets)
integerMapping=dict(zip(targets,integerEncoded))
integerMapping

{1: 0}

In [22]:
targets = df1['actual_class']   
integerEncoded = le.fit_transform(targets)
integerMapping=dict(zip(targets,integerEncoded))
integerMapping

{2: 2, 1: 1, 0: 0, 3: 3}

**Cek apakah prediksi class benar**

In [23]:
df1['check'] = df1['label_actual'] - df1['label_pred']
df1.head(6)

,1,2,4,sum,predicted_class,actual_class,label_pred,label_actual,check
0,0.809970,0.169699,0.020331,1.000000,1,2,0,2,2
1,0.994739,0.002403,0.002858,1.000000,1,1,0,1,1
2,0.927942,0.016056,0.056002,1.000000,1,1,0,1,1
3,0.947341,0.005885,0.046774,1.000000,1,1,0,1,1
4,0.999420,0.000553,0.000027,1.000000,1,0,0,0,0
5,0.912600,0.011202,0.076198,1.000000,1,1,0,1,1


In [24]:
df1['correct_prediction?'] = np.where(df1['check'] == 0, 'True', 'False')
df1 = df1.drop(['label_pred', 'label_actual', 'check'], axis=1)
df1.head()

,1,2,4,sum,predicted_class,actual_class,correct_prediction?
0,0.809970,0.169699,0.020331,1.000000,1,2,False
1,0.994739,0.002403,0.002858,1.000000,1,1,False
2,0.927942,0.016056,0.056002,1.000000,1,1,False
3,0.947341,0.005885,0.046774,1.000000,1,1,False
4,0.999420,0.000553,0.000027,1.000000,1,0,True


In [25]:
display(df1)

,1,2,4,sum,predicted_class,actual_class,correct_prediction?
0,0.809970,0.169699,0.020331,1.000000,1,2,False
1,0.994739,0.002403,0.002858,1.000000,1,1,False
2,0.927942,0.016056,0.056002,1.000000,1,1,False
3,0.947341,0.005885,0.046774,1.000000,1,1,False
4,0.999420,0.000553,0.000027,1.000000,1,0,True
5,0.912600,0.011202,0.076198,1.000000,1,1,False
6,0.999914,0.000082,0.000004,1.000000,1,3,False
7,0.936033,0.021538,0.042429,1.000000,1,1,False
8,0.959194,0.015165,0.025641,1.000000,1,1,False


In [26]:
true_predictions = df1[(df1["correct_prediction?"] == 'True')].shape[0]
false_predictions = df1[(df1["correct_prediction?"] == 'False')].shape[0]
total = df1["correct_prediction?"].shape[0]

print('manual calculated Accuracy is:', (true_predictions / total * 100))

manual calculated Accuracy is: 11.11111111111111


In [27]:
wrong_pred = df1[(df1["correct_prediction?"] == 'False')]
wrong_pred

,1,2,4,sum,predicted_class,actual_class,correct_prediction?
0,0.809970,0.169699,0.020331,1.000000,1,2,False
1,0.994739,0.002403,0.002858,1.000000,1,1,False
2,0.927942,0.016056,0.056002,1.000000,1,1,False
3,0.947341,0.005885,0.046774,1.000000,1,1,False
5,0.912600,0.011202,0.076198,1.000000,1,1,False
6,0.999914,0.000082,0.000004,1.000000,1,3,False
7,0.936033,0.021538,0.042429,1.000000,1,1,False
8,0.959194,0.015165,0.025641,1.000000,1,1,False


***
# 4. Kesimpulan
***

-  Regresi Logistik Multinomial dapat digunakan untuk memprediksi beberapa kelas
-  Dengan data yang ada, dilakukan validasi dengan testing size 30% menghasilkan akurasi data 88,88% 